<a href="https://colab.research.google.com/github/akimovhhh/predatory_pricing_airlines/blob/main/notebooks/Data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- install Chromium + chromedriver + Selenium ---
!apt-get update -qq
!apt-get install -y chromium-browser chromium-chromedriver
!pip install -q selenium

# --- mount your Google Drive ---
# Comment this lines if run locally
from google.colab import drive
drive.mount("/content/drive", force_remount=False)


In [ ]:
import pathlib
import time
import zipfile
import contextlib
import sys
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

YEAR_DROPDOWN_ID = "cboYear"     # <select> with the years
QUARTER_DROPDOWN_ID = "cboPeriod"  # <select> with the quarters (ignored if QUARTERS=[])
DOWNLOAD_BTN_ID  = "btnDownload" # "Download" button
CHECKBOX_ID      = "chkAllVars"  # "Select all variables" (used if REQUIRED_FIELDS=[])

FIRST_YEAR, LAST_YEAR = 2005, 2025    # inclusive loop
TIMEOUT_SEC           = 600            # increased timeout
HEADLESS              = True          # set False to watch the browser
DOWNLOAD_WAIT_SEC     = 600            # max time to wait for download
OVERWRITE_FILES       = "overwrite"        # "overwrite", "timestamp", or "skip"
REFRESH_EVERY_N_DOWNLOADS = 4         # refresh page every N downloads to avoid popups

#  Google Drive paths - works for both T100D and DB1B datasets
# Change to your own path, but keep in mind that data is almost 150 GB
GDRIVE_MYDRIVE = pathlib.Path("/content/drive/My Drive")

In [ ]:
def make_driver() -> webdriver.Chrome:
    """Create Chrome driver with proper download settings"""
    opts = webdriver.ChromeOptions()
    opts.binary_location = "/usr/bin/chromium-browser"

    if HEADLESS:
        opts.add_argument("--headless")

    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--disable-software-rasterizer")

    prefs = {
        "download.default_directory": str(ZIP_DIR.absolute()),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "profile.default_content_settings.popups": 0,
        "profile.default_content_setting_values.automatic_downloads": 1
    }
    opts.add_experimental_option("prefs", prefs)       # <-- you were missing this

    driver = webdriver.Chrome(options=opts)

    # Enable downloads in headless mode
    driver.execute_cdp_cmd(
        "Page.setDownloadBehavior",
        {"behavior": "allow", "downloadPath": str(ZIP_DIR.absolute())}
    )

    return driver


def wait_for_download_complete(initial_files: set, max_wait_sec: int = DOWNLOAD_WAIT_SEC) -> pathlib.Path | None:
    """
    Wait for a new file to appear in the download directory.
    Returns the path to the new file, or None if timeout.
    """
    start_time = time.time()

    while time.time() - start_time < max_wait_sec:
        # Check for any .crdownload files (incomplete downloads)
        crdownload_files = list(ZIP_DIR.glob("*.crdownload"))
        if crdownload_files:
            print(f"Download in progress... ({len(crdownload_files)} files)")
            time.sleep(2)
            continue

        # Check for new files
        current_files = set(ZIP_DIR.iterdir())
        new_files = current_files - initial_files

        # Filter for actual files (not directories) and reasonable size
        new_data_files = [
            f for f in new_files
            if f.is_file() and f.stat().st_size > 1000  # at least 1KB
        ]

        if new_data_files:
            # Return the newest file
            newest = max(new_data_files, key=lambda p: p.stat().st_mtime)
            print(f"✓ Download complete: {newest.name} ({newest.stat().st_size} bytes)")
            return newest

        time.sleep(1)

    print(f"⚠ Download timeout after {max_wait_sec} seconds")
    return None

def dismiss_popups(driver, wait):
    """Try to dismiss any survey popups or overlays that might be blocking the page"""
    try:
        # Common popup selectors to try
        popup_selectors = [
            "div[id*='QSIPopOver']",  # Qualtrics popup
            "div[class*='popup']",
            "div[class*='overlay']",
            "div[class*='modal']",
            "button[id*='close']",
            "button[class*='close']",
            "img[src*='qualtrics']",
            "[aria-label*='close']",
            "[aria-label*='Close']"
        ]

        dismissed = False
        for selector in popup_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    if element.is_displayed():
                        print(f"    🚫 Dismissing popup: {selector}")
                        element.click()
                        dismissed = True
                        time.sleep(1)
                        break
            except Exception:
                continue

        # Try pressing ESC key to close popups
        if not dismissed:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
                print("    🚫 Pressed ESC to dismiss popup")
                time.sleep(1)
            except Exception:
                pass

    except Exception as e:
        print(f"    ⚠ Popup dismissal failed: {e}")

def select_data_fields(driver, wait):
    """Select data fields - either use 'select all' checkbox or individual fields"""

    # Method 1: Try "select all" checkbox first (if REQUIRED_FIELDS is empty)
    if not REQUIRED_FIELDS:
        try:
            print("  📋 Using 'Select All' checkbox...")
            chk = wait.until(EC.element_to_be_clickable((By.ID, CHECKBOX_ID)))
            if not chk.is_selected():
                chk.click()
                print("  ✓ All fields selected via checkbox")
            else:
                print("  ✓ All fields already selected")
        except Exception as e:
            print(f"  ⚠ Select all checkbox failed: {e}")
            print("  🔄 Falling back to manual field selection...")
            # Continue to manual selection below
        else:
            # Successfully used select all, now check for ZIP download option
            try:
                zip_checkbox = driver.find_element(By.ID, "chkDownloadZip")
                if not zip_checkbox.is_selected():
                    zip_checkbox.click()
                    print("  ✓ ZIP download option selected")
                else:
                    print("  ✓ ZIP download already selected")
            except Exception as e:
                print(f"  ⚠ ZIP download checkbox not found: {e}")

            return True, []

    # Method 2: Manual field selection
    print(f"  📋 Selecting {len(REQUIRED_FIELDS)} required data fields...")

    selected_count = 0
    missing_fields = []

    for field_name in REQUIRED_FIELDS:
        try:
            # Try different possible checkbox patterns for this field
            possible_selectors = [
                f"input[value='{field_name}']",
                f"input[name*='{field_name}']",
                f"input[id*='{field_name}']",
                f"//input[@value='{field_name}']",
                f"//input[contains(@name, '{field_name}')]",
                f"//input[contains(@id, '{field_name}')]"
            ]

            field_found = False
            for selector in possible_selectors:
                try:
                    if selector.startswith("//"):
                        # XPath selector
                        element = driver.find_element(By.XPATH, selector)
                    else:
                        # CSS selector
                        element = driver.find_element(By.CSS_SELECTOR, selector)

                    if not element.is_selected():
                        # Scroll to element and click
                        driver.execute_script("arguments[0].scrollIntoView(true);", element)
                        time.sleep(0.1)
                        element.click()

                    selected_count += 1
                    field_found = True
                    print(f"    ✓ {field_name}")
                    break
                except Exception:
                    continue

            if not field_found:
                missing_fields.append(field_name)
                print(f"    ❌ {field_name} (not found)")

        except Exception as e:
            missing_fields.append(field_name)
            print(f"    ❌ {field_name} (error: {str(e)[:50]})")

    print(f"  📊 Fields selected: {selected_count}/{len(REQUIRED_FIELDS)}")
    if missing_fields:
        print(f"  ⚠ Missing fields: {missing_fields}")

    # After manual field selection, also check for ZIP download option
    if selected_count > 0:
        try:
            zip_checkbox = driver.find_element(By.ID, "chkDownloadZip")
            if not zip_checkbox.is_selected():
                zip_checkbox.click()
                print("  ✓ ZIP download option selected")
            else:
                print("  ✓ ZIP download already selected")
        except Exception as e:
            print(f"  ⚠ ZIP download checkbox not found: {e}")

    return selected_count > 0, missing_fields

def click_download_button_robustly(driver, wait):
    """Try multiple methods to click the download button, handling popups and overlays"""
    button_clicked = False

    # Method 1: Try normal click after dismissing popups
    try:
        dismiss_popups(driver, wait)
        dl_btn = wait.until(EC.element_to_be_clickable((By.ID, DOWNLOAD_BTN_ID)))
        dl_btn.click()
        button_clicked = True
        print("    ✓ Download button clicked (normal)")
    except Exception as e:
        print(f"    ⚠ Normal click failed: {str(e)[:100]}...")

    # Method 2: Try JavaScript click if normal click failed
    if not button_clicked:
        try:
            dl_btn = driver.find_element(By.ID, DOWNLOAD_BTN_ID)
            driver.execute_script("arguments[0].click();", dl_btn)
            button_clicked = True
            print("    ✓ Download button clicked (JavaScript)")
        except Exception as e:
            print(f"    ⚠ JavaScript click failed: {str(e)[:100]}...")

    # Method 3: Try scrolling to button and clicking
    if not button_clicked:
        try:
            dl_btn = driver.find_element(By.ID, DOWNLOAD_BTN_ID)
            driver.execute_script("arguments[0].scrollIntoView(true);", dl_btn)
            time.sleep(1)
            dl_btn.click()
            button_clicked = True
            print("    ✓ Download button clicked (scroll + click)")
        except Exception as e:
            print(f"    ⚠ Scroll + click failed: {str(e)[:100]}...")

    if not button_clicked:
        raise Exception("All download button click methods failed")

    return button_clicked

def is_valid_zip(file_path: pathlib.Path) -> bool:
    """Check if a file is a valid ZIP archive"""
    try:
        with zipfile.ZipFile(file_path, 'r') as z:
            # Try to read the file list
            z.namelist()
            return True
    except zipfile.BadZipFile:
        return False
    except Exception as e:
        print(f"Error checking ZIP file: {e}")
        return False

def unzip_to_drive(year: int, quarter: int = None, zip_path: pathlib.Path = None, overwrite: str = "skip") -> bool:
    """Extract ZIP file to the raw directory with appropriate prefixed filenames"""
    try:
        # Extract to the main raw directory (no subdirectories)
        RAW_DIR.mkdir(parents=True, exist_ok=True)

        with zipfile.ZipFile(zip_path, "r") as z:
            file_list = z.namelist()

            # Determine file prefix based on whether we have quarters
            if quarter is not None:
                prefix = f"{year}_Q{quarter}"
                print(f"  Extracting {len(file_list)} files with prefix {prefix}")
            else:
                prefix = f"{year}"
                print(f"  Extracting {len(file_list)} files with prefix {prefix}")

            extracted_count = 0
            skipped_count = 0

            for file_name in file_list:
                # Skip directories
                if file_name.endswith('/'):
                    continue

                # Extract file to temporary location first
                z.extract(file_name, RAW_DIR)

                # Get the original file path
                original_path = RAW_DIR / file_name

                # Create new filename with appropriate prefix
                file_stem = original_path.stem  # filename without extension
                file_suffix = original_path.suffix  # file extension
                new_filename = f"{prefix}_{file_stem}{file_suffix}"
                new_path = RAW_DIR / new_filename

                # Handle existing files based on overwrite setting
                if new_path.exists():
                    if overwrite == "overwrite":
                        print(f"    ⚠ Overwriting: {new_filename}")
                        new_path.unlink()  # Remove existing file
                    elif overwrite == "timestamp":
                        # Create a unique filename with timestamp
                        import datetime
                        timestamp = datetime.datetime.now().strftime("%H%M%S")
                        new_filename = f"{prefix}_{file_stem}_{timestamp}{file_suffix}"
                        new_path = RAW_DIR / new_filename
                        print(f"    📝 Creating timestamped: {new_filename}")
                    elif overwrite == "skip":
                        print(f"    ⏭ Skipping existing: {new_filename}")
                        original_path.unlink()  # Remove the temporary extracted file
                        skipped_count += 1
                        continue

                # Rename the file
                original_path.rename(new_path)
                print(f"    → {new_filename}")
                extracted_count += 1

                # Clean up any empty directories created during extraction
                try:
                    if original_path.parent != RAW_DIR and original_path.parent.exists():
                        original_path.parent.rmdir()
                except OSError:
                    pass  # Directory not empty, that's fine

            print(f"  📊 Extracted: {extracted_count}, Skipped: {skipped_count}")
        return True
    except Exception as e:
        print(f"Error extracting {zip_path}: {e}")
        return False

def main():
    driver = None
    try:
        # Determine data type based on configuration
        is_quarterly = len(QUARTERS) > 0
        data_type = "quarterly" if is_quarterly else "annual"

        print(f"Starting BTS {data_type} data download...")
        if is_quarterly:
            print("Dataset: DB1B - Origin and Destination Survey (Quarterly)")
        else:
            print("Dataset: T100D - Domestic Segment Data (Annual)")

        driver = make_driver()
        wait = WebDriverWait(driver, TIMEOUT_SEC)

        print(f"Navigating to: {START_URL}")
        driver.get(START_URL)

        # Wait for page to load
        try:
            wait.until(EC.presence_of_element_located((By.ID, YEAR_DROPDOWN_ID)))
            print("✓ Page loaded successfully")
        except TimeoutException:
            print("❌ Page failed to load properly")
            return

        downloads_processed = 0

        # Handle both quarterly and annual data
        quarters_to_process = QUARTERS if is_quarterly else [None]  # None = no quarter selection

        for yr in range(FIRST_YEAR, LAST_YEAR + 1):
            for qtr in quarters_to_process:

                # Display progress message
                if qtr is not None:
                    print(f"\n📥 Processing {yr} Q{qtr}...")
                    download_id = f"{yr}_Q{qtr}"
                else:
                    print(f"\n📥 Processing {yr}...")
                    download_id = f"{yr}"

                # Refresh page every few downloads to avoid popup accumulation
                if downloads_processed > 0 and downloads_processed % REFRESH_EVERY_N_DOWNLOADS == 0:
                    print("  🔄 Refreshing page to avoid popups...")
                    driver.get(START_URL)
                    try:
                        wait.until(EC.presence_of_element_located((By.ID, YEAR_DROPDOWN_ID)))
                        print("  ✓ Page refreshed successfully")
                    except TimeoutException:
                        print("  ❌ Page refresh failed")
                        continue

                # Record files before download
                initial_files = set(ZIP_DIR.iterdir())

                try:
                    # 1️⃣ Select the year
                    print(f"  Selecting year {yr}")
                    year_dropdown = Select(driver.find_element(By.ID, YEAR_DROPDOWN_ID))
                    year_dropdown.select_by_visible_text(str(yr))
                    time.sleep(1)  # Small delay after selection

                    # 2️⃣ Select the quarter (only if quarterly data)
                    if qtr is not None:
                        print(f"  Selecting quarter {qtr}")
                        try:
                            quarter_dropdown = Select(driver.find_element(By.ID, QUARTER_DROPDOWN_ID))
                            quarter_dropdown.select_by_visible_text(f"Quarter {qtr}")
                            time.sleep(1)  # Small delay after selection
                        except Exception as e:
                            print(f"  ⚠ Quarter selection failed: {e}")
                            continue

                    # 3️⃣ Select data fields (either "select all" or manual)
                    fields_selected, missing_fields = select_data_fields(driver, wait)
                    if not fields_selected:
                        print(f"❌ No fields selected for {download_id}")
                        continue

                    # 4️⃣ Click Download (with popup handling)
                    print("  Clicking download button")
                    click_download_button_robustly(driver, wait)

                    # 5️⃣ Wait for download to complete
                    print("  Waiting for download to complete...")
                    downloaded_file = wait_for_download_complete(initial_files)

                    if downloaded_file is None:
                        print(f"❌ Download failed for {download_id}")
                        continue

                    # 6️⃣ Verify it's a valid ZIP file
                    if not is_valid_zip(downloaded_file):
                        print(f"❌ Downloaded file is not a valid ZIP: {downloaded_file}")
                        continue

                    # 7️⃣ Rename to a clean name
                    clean_name = ZIP_DIR / f"{download_id}.zip"
                    if clean_name.exists():
                        clean_name.unlink()
                    downloaded_file.rename(clean_name)
                    print(f"  Renamed to: {clean_name.name}")

                    # 8️⃣ Extract the ZIP file (with appropriate naming)
                    print("  Extracting files...")
                    if unzip_to_drive(yr, qtr, clean_name, overwrite=OVERWRITE_FILES):
                        print(f"✅ {download_id} completed successfully")
                        downloads_processed += 1
                    else:
                        print(f"❌ Extraction failed for {download_id}")

                except TimeoutException as e:
                    print(f"❌ Timeout error for {download_id}: {e}")
                    # Try refreshing page on timeout
                    print("  🔄 Attempting page refresh due to timeout...")
                    try:
                        driver.get(START_URL)
                        wait.until(EC.presence_of_element_located((By.ID, YEAR_DROPDOWN_ID)))
                    except Exception:
                        print("  ❌ Recovery refresh failed")
                except Exception as e:
                    print(f"❌ Error processing {download_id}: {e}")
                    # Try refreshing page on error
                    if "click intercepted" in str(e) or "popup" in str(e).lower():
                        print("  🔄 Popup detected, refreshing page...")
                        try:
                            driver.get(START_URL)
                            wait.until(EC.presence_of_element_located((By.ID, YEAR_DROPDOWN_ID)))
                        except Exception:
                            print("  ❌ Recovery refresh failed")

    except Exception as e:
        print(f"❌ Fatal error: {e}")
    finally:
        if driver:
            driver.quit()
            print("\n🔒 Browser closed")

    print(f"\n📋 SUMMARY:")
    print(f"  Data type: {data_type}")
    print(f"  Downloads processed: {downloads_processed}")
    print(f"  ZIP files: {ZIP_DIR}")
    print(f"  CSV files: {RAW_DIR}")

    # Show what we actually downloaded
    zip_files = list(ZIP_DIR.glob("*.zip"))
    csv_files = list(RAW_DIR.glob("*.csv"))

    if zip_files:
        print(f"  Successfully downloaded: {[f.name for f in zip_files]}")
    else:
        print("  ⚠ No ZIP files found")

    if csv_files:
        print(f"  CSV files created: {[f.name for f in csv_files]}")
    else:
        print("  ⚠ No CSV files found")

In [ ]:
# FOR T100D (Annual Data):
DRIVE_ROOT = GDRIVE_MYDRIVE / "predatory_pricing_in_airlines" / "data" / "T100"
ZIP_DIR  = DRIVE_ROOT / "zip"
RAW_DIR  = DRIVE_ROOT / "raw"
ZIP_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)
START_URL = "https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=GEE&QO_fu146_anzr=Nv4+Pn44vr45"
QUARTERS = []  # Empty = annual data
REQUIRED_FIELDS = []  # Empty = use "select all" button
CHECKBOX_ID = "chkAllVars"

try:
  main()
except KeyboardInterrupt:
  print("\n⏹ Download interrupted by user")
except Exception as e:
  print(f"\n💥 Unexpected error: {e}")
  sys.exit(1)

In [ ]:
# FOR DB1B (Quarterly Data):
DRIVE_ROOT = GDRIVE_MYDRIVE / "predatory_pricing_in_airlines" / "data" / "DB1B"
ZIP_DIR  = DRIVE_ROOT / "zip"
RAW_DIR  = DRIVE_ROOT / "raw"
ZIP_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)
START_URL = "https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FHK&QO_fu146_anzr=b4vtv0+n0q+Qr56v0n6v10+f748rB"
QUARTERS = [1, 2, 3, 4]  # [1,2,3,4] = quarterly, [] = annual
REQUIRED_FIELDS = [  # Empty [] = use "select all", filled = manual selection
    "ITIN_ID", "MKT_ID", "MARKET_COUPONS", "YEAR", "QUARTER", "ORIGIN", "ORIGIN_COUNTRY",
    "ORIGIN_STATE_FIPS", "ORIGIN_STATE_ABR", "ORIGIN_STATE_NM", "ORIGIN_WAC", "DEST",
    "DEST_COUNTRY", "DEST_STATE_FIPS", "DEST_STATE_ABR", "DEST_STATE_NM", "DEST_WAC",
    "AIRPORT_GROUP", "WAC_GROUP", "TK_CARRIER_CHANGE", "TK_CARRIER_GROUP",
    "OP_CARRIER_CHANGE", "OP_CARRIER_GROUP", "REPORTING_CARRIER", "TICKET_CARRIER",
    "OPERATING_CARRIER", "BULK_FARE", "PASSENGERS", "MARKET_FARE", "MARKET_DISTANCE",
    "DISTANCE_GROUP", "MARKET_MILES_FLOWN", "NONSTOP_MILES", "ITIN_GEO_TYPE", "MKT_GEO_TYPE"
]

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n⏹ Download interrupted by user")
    except Exception as e:
        print(f"\n💥 Unexpected error: {e}")
        sys.exit(1)